<a href="https://colab.research.google.com/github/EricCallaway/COSC_5313_CNN_model_HW-5/blob/test/CNN_model_HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import pandas as pd
import numpy as np



Import Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
img_data_path_1 = "/content/drive/MyDrive/Data/HW#5_Dataset/imgs1/"
img_data_path_2 = "/content/drive/MyDrive/Data/HW#5_Dataset/imgs2/"
file_data = "/content/drive/MyDrive/Data/HW#5_Dataset/file_data.csv"
class_0_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_0.csv"
class_1_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_1.csv"
class_2_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_2.csv"
class_3_path = "/content/drive/MyDrive/Data/HW#5_Dataset/class_3.csv"


df = pd.read_csv(file_data)
df_class_0 = pd.read_csv(class_0_path)
df_class_1 = pd.read_csv(class_1_path)
df_class_2 = pd.read_csv(class_2_path)
df_class_3 = pd.read_csv(class_3_path)


seq_nums = df['seq_num'].values
file_name = df[' file_name'].values
class_label = df[' class_label'].values



In [4]:
# pip install split-folders

In [5]:
# import splitfolders
# input = '/content/drive/MyDrive/Data/HW#5_Dataset/imgs1'
# output = 'final_dataset'
# splitfolders.ratio(input, output=output, seed=1337, ratio=(.7, 0.15,0.15))

In [6]:
train_path = '/content/drive/MyDrive/Data/HW#5_Dataset/final_dataset/train/'
test_path = '/content/drive/MyDrive/Data/HW#5_Dataset/final_dataset/test/'
val_path = '/content/drive/MyDrive/Data/HW#5_Dataset/final_dataset/val/'

In [7]:
HEIGHT = 256
WIDTH = 256

In [8]:
import os
import cv2
x_train = []
for folder in os.listdir(train_path):
  sub_path = train_path + folder
  for img in os.listdir(sub_path):
    image_path = sub_path + '/' + img
    img_arr = cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(HEIGHT,WIDTH))
    x_train.append(img_arr)



In [9]:
print(len(x_train))

3043


In [10]:
x_test = []
for folder in os.listdir(test_path):
  sub_path = test_path + folder
  for img in os.listdir(sub_path):
    image_path = sub_path + '/' + img
    img_arr = cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(HEIGHT,WIDTH))
    x_test.append(img_arr)

In [11]:
print(len(x_test))

655


In [12]:
x_val = []
for folder in os.listdir(val_path):
  sub_path = val_path + folder
  for img in os.listdir(sub_path):
    image_path = sub_path + '/' + img
    img_arr = cv2.imread(image_path)
    img_arr=cv2.resize(img_arr,(HEIGHT,WIDTH))
    x_val.append(img_arr)

In [13]:
print(len(x_val))

651


Turn our Train, Test, and Validation lists into numpy arrays.
Also normalize the values in the arrays between the values 0 and 1. We do this by dividing each element in the array by 255.

In [14]:
train_x = np.array(x_train)
test_x = np.array(x_test)
val_x = np.array(x_val)

train_x = train_x/255.0
test_x = test_x/255.0
val_x = val_x/255.0

Compute labels of corresponding datasets using ImgageDataGenerator. We use this because our images are stored in folders. We must walk through the folders and find out the corresoponding labels of images stored here. 

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (HEIGHT, WIDTH),
                                                 batch_size = 32,
                                                 class_mode = 'sparse',
                                                 classes={'normal': 0, 'glue_defects': 1, 'bubble_defects': 2, 'object_defects': 3})
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (HEIGHT, WIDTH),
                                            batch_size = 32,
                                            class_mode = 'sparse',
                                            classes={'normal': 0, 'glue_defects': 1, 'bubble_defects': 2, 'object_defects': 3})
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (HEIGHT, WIDTH),
                                            batch_size = 32,
                                            class_mode = 'sparse',
                                            classes={'normal': 0, 'glue_defects': 1, 'bubble_defects': 2, 'object_defects': 3})

Found 3043 images belonging to 4 classes.
Found 655 images belonging to 4 classes.
Found 651 images belonging to 4 classes.


In [17]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

This shows the indicies of each class.

In [18]:
val_set.class_indices

{'bubble_defects': 2, 'glue_defects': 1, 'normal': 0, 'object_defects': 3}

In [19]:
test_set.class_indices

{'bubble_defects': 2, 'glue_defects': 1, 'normal': 0, 'object_defects': 3}

In [20]:
training_set.class_indices

{'bubble_defects': 2, 'glue_defects': 1, 'normal': 0, 'object_defects': 3}

Shape of each set. Notice they are all 1D arrays

In [21]:
train_y.shape

(3043,)

In [22]:
test_y.shape

(655,)

In [23]:
val_y.shape

(651,)

Build the Model

In [24]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(HEIGHT,WIDTH,3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 activation (Activation)     (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 254, 254, 32)      9248      
                                                                 
 activation_1 (Activation)   (None, 254, 254, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 32)      0         
                                                        

Applying Early Stopping to avoid overfitting of the model

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

Fit the Model

In [ ]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

Epoch 1/10
